In [1]:
suppressMessages(library(tidyverse))
suppressMessages(library(reshape2))
suppressMessages(library(gridExtra))
suppressMessages(library(grid))
suppressMessages(library(rstatix))
suppressMessages(library(ggpubr))

suppressMessages(source("../0_support-files/theme_CRP-MISC.R"))

protein_rds <- readRDS("../1_sample-data/protein_val_date.rds")

CELLTYPES <- c('Intrahepatic.cholangiocyte','NK.Cell','Monocyte','Macrophage','Secretory.cell','Respiratory.ciliated.cell',
               'Ionocyte.luminal.epithelial.cell.of.mammary.gland','Fibroblast.mesenchymal.stem.cell','Respiratory.secretory.cell',
               'Mast.cell','Basal.cell','Endothelial.cell','neutrophil','B.cell','T.Cell','Erythrocyte.erythroid.progenitor','Platelet',
               'Basophil','Stromal.cell','Mature.conventional.dendritic.cell','Adventitial.cell','Gland.cell','Salivary.bronchial.secretory.cell',
               'Pericyte.cell','Myeloid.progenitor','Pancreatic.Stellate.cell','Pancreatic.alpha.beta.cell','Basal.prostate.cell',
               'Prostate.epithelia','Salivary.gland.cell','Intestinal.enterocyte','Intestinal.secretory.cell','Intestinal.tuft.cell',
               'Type.II.Pneumocyte','Cell.of.skeletal.muscle','Schwann.cell','Tendon.cell','Mesothelial.cell','Plasmablast',
               'Kidney.epithelial.cell','Thymocyte')

---
# cfRNA and wbRNA data

### cfRNA

In [2]:
##------------------------------------
# META DATA
meta_data_too <- read.csv("../1_sample-data/STable6_cfrna-samples.csv") %>%  
    mutate(PTID_TIME = paste0(PTID,"_",timepoint)) #%>%
    # filter(cfrna_sample_id %in% all_of(paired$cfrna_sample_id)) %>%
    # filter(timepoint == "acute")

# cf_decon <- cf_meta[,c("cfrna_sample_id",CELLTYPES)]%>%
#     filter(cfrna_sample_id %in% all_of(cf_meta$cfrna_sample_id))

# ##------------------------------------
# # MERGE
# meta_data_too = merge(cf_meta,cf_decon,by="cfrna_sample_id")

##------------------------------------
# MELT

IDVARS = colnames(meta_data_too)
IDVARS = IDVARS[!(IDVARS %in% CELLTYPES)]
df_melt <- melt(meta_data_too,id.vars=IDVARS)
i = length(IDVARS)
colnames(df_melt)[(i+1):(i+2)] <- c("celltype","fraction")


##---------------------------
# Relabel cell types

group = list("Myeloid Progenitor" = c("Myeloid.progenitor"),
             "Erythrocyte/Erythroid Progenitor" = c("Erythrocyte.erythroid.progenitor"),
             "Intestinal Epithelial" = c("Intestinal.tuft.cell","Intestinal.enterocyte"),
             "T Cell / NK Cell" = c("T.Cell","NK.Cell"),
            "Kidney Epithelial Cell" = c("Kidney.epithelial.cell"),
            "Endothelial Cell" = c("Endothelial.cell"),
             "Neutrophil" = c("neutrophil"),
             "Platelet" = c("Platelet"),
             "Monocyte" = c("Monocyte"),
             "Thymocyte" = c("Thymocyte"),
             'Intrahepatic cholangiocyte' = c('Intrahepatic.cholangiocyte')
            )

# create dataframe
df2 <- stack(group) 
colnames(df2) <- c("celltype","new_celltype")

# merge and fix nas
df_melt2 <- merge(df_melt,df2,all.x=TRUE)
df_melt2$new_celltype <- as.character(df_melt2$new_celltype)
df_melt_all <- df_melt2                                                     ### SAVE FOR LATER PLOTTING
df_melt2[is.na(df_melt2$new_celltype),"new_celltype"] <- "other"

# Save as main table
df_melt2$celltype <- df_melt2$new_celltype
df_melt2 <- df_melt2 %>% group_by(across(c(-fraction))) %>% summarize(fraction = sum(fraction))

df_melt <- df_melt2 %>% ungroup()


##------------------------------------
# MODIFY IDs
df_melt <- df_melt %>%
    select(cfrna_sample_id,Diagnosis,timepoint,PTID,severity,celltype,fraction) %>%
    mutate(PTID = toupper(PTID))

df_melt_cf <- df_melt

df_melt_cf <- df_melt_cf %>% select(!PTID)



`summarise()` has grouped output by 'celltype', 'PTID', 'cfrna_sample_id', 'group', 'Diagnosis', 'severity', 'timepoint', 'reads_sequenced', 'feature_counts', 'intron_exon_ratio', 'bias53',
'rRNA_reads', 'ivig', 'days_ivig_before_samp', 'ivig_rel_samp', 'PTID_TIME'. You can override using the `.groups` argument.


### wbrna

In [3]:
##------------------------------------
# META DATA
wb_meta <- read.csv("../1_sample-data/STable7_wbrna-samples.csv") %>% 
    mutate(PTID_TIME = paste0(PTID,"_",timepoint))# %>%
    # filter(wbrna_sample_id %in% all_of(paired$wbrna_sample_id))%>%
    # filter(timepoint == "acute")


##------------------------------------
# MERGE
meta_data_too = wb_meta

##------------------------------------
# MELT

IDVARS = colnames(wb_meta)
IDVARS = IDVARS[!(IDVARS %in% CELLTYPES)]
df_melt <- melt(meta_data_too,id.vars=IDVARS)
i = length(IDVARS)
colnames(df_melt)[(i+1):(i+2)] <- c("celltype","fraction")


##---------------------------
# Relabel cell types

group = list("Myeloid Progenitor" = c("Myeloid.progenitor"),
             "Erythrocyte/Erythroid Progenitor" = c("Erythrocyte.erythroid.progenitor"),
             "Intestinal Epithelial" = c("Intestinal.tuft.cell","Intestinal.enterocyte"),
             "T Cell / NK Cell" = c("T.Cell","NK.Cell"),
            "Kidney Epithelial Cell" = c("Kidney.epithelial.cell"),
            "Endothelial Cell" = c("Endothelial.cell"),
             "Neutrophil" = c("neutrophil"),
             "Platelet" = c("Platelet"),
             "Monocyte" = c("Monocyte"),
             "Thymocyte" = c("Thymocyte"),
             'Intrahepatic cholangiocyte' = c('Intrahepatic.cholangiocyte')
            )

# create dataframe
df2 <- stack(group) 
colnames(df2) <- c("celltype","new_celltype")

# merge and fix nas
df_melt2 <- merge(df_melt,df2,all.x=TRUE)
df_melt2$new_celltype <- as.character(df_melt2$new_celltype)
df_melt_all <- df_melt2                                                     ### SAVE FOR LATER PLOTTING
df_melt2[is.na(df_melt2$new_celltype),"new_celltype"] <- "other"

# Save as main table
df_melt2$celltype <- df_melt2$new_celltype
df_melt2 <- df_melt2 %>% group_by(across(c(-fraction))) %>% summarize(fraction = sum(fraction))

df_melt <- df_melt2 %>% ungroup()

##------------------------------------
# MODIFY IDs
df_melt <- df_melt %>%
    select(wbrna_sample_id,Diagnosis,timepoint,PTID,severity,celltype,fraction) %>%
    mutate(PTID = toupper(PTID))

df_melt_wb <- df_melt

df_melt_wb <- df_melt_wb %>% select(!PTID)



`summarise()` has grouped output by 'celltype', 'PTID', 'wbrna_sample_id', 'group', 'Diagnosis', 'severity', 'timepoint', 'reads_sequenced', 'transcriptome_cov', 'ivig',
'days_ivig_before_samp', 'ivig_rel_samp', 'PTID_TIME'. You can override using the `.groups` argument.


---
# Saved Variables

### palettes

In [5]:
##---------------------------
# New palette - cfRNA

cf_palette = c('#0234C1','#740C33','#4b7a47','#FB927F','#5CFBF4','#DBB008','#86C35E','#4D5D0D','#2BBBB3','#B27DF2','#1362AB','#FFCCCB')

## REORDER BASED ON PREVELANCE
sum <- df_melt_cf %>% 
    filter(Diagnosis == "MIS-C" & timepoint == "acute") %>% 
    group_by(celltype) %>% summarize(mean=mean(fraction))

sum <- sum[order(sum$mean,decreasing=TRUE),]

names(cf_palette) <- sum$celltype

##---------------------------
# New palette - wbRNA

library(qualpalr)
num_cells = length(unique(df_melt_wb$celltype))
wb_palette = sample(rev(qualpal(num_cells, colorspace=list(h=c(0,360), s=c(0.3,1), l=c(0.2,0.8)))$hex))

## REORDER BASED ON PREVELANCE
sum <- df_melt_wb %>% 
    filter(Diagnosis == "MIS-C" & timepoint == "acute") %>% 
    group_by(celltype) %>% summarize(mean=mean(fraction))

sum <- sum[order(sum$mean,decreasing=TRUE),]

names(wb_palette) <- sum$celltype

##---------------------------
# New palette - wbRNA
group_palette = c("low" = "#FFF200", "high" = "#2E3192")

### functions

In [6]:
get_plot_df <- function(mdf, df_melt, MARKER, CUTOFF, ID_VAR, DATE_THRESH = 4){

    if (mdf[1,"date_diff"] != "NO DATE" | is.na(mdf[1,"date_diff"])){
        
        mdf <- mdf %>% arrange(.data[[ID_VAR]],date_diff)
        mdf <- mdf[!duplicated(mdf[[ID_VAR]]),]
        
        # filter samples
        mdf_pb = mdf %>%  
            filter(date_diff <= DATE_THRESH) %>%
            select(.data[[ID_VAR]], .data[[MARKER]])
        
        mdf_pb = merge(mdf_pb, df_melt, by=ID_VAR) %>%
            filter(!is.na(.data[[MARKER]])) %>%
            filter(.data[[MARKER]] != "")
            
        mdf_pb[[MARKER]] <- as.numeric(mdf_pb[[MARKER]])
    
    } else{
        mdf <- mdf %>% arrange(.data[[ID_VAR]],desc(.data[[MARKER]]))
        mdf <- mdf[!duplicated(mdf[[ID_VAR]]),]
        
        # filter samples
        mdf_pb = mdf %>% 
            select(.data[[ID_VAR]], .data[[MARKER]]) %>% 
            filter(!is.na(.data[[MARKER]])) %>%
            filter(.data[[MARKER]] != "")

        mdf_pb[[MARKER]] <- as.numeric(mdf_pb[[MARKER]])

        # merge w/ TOO
        mdf_pb <- merge(mdf_pb, df_melt, by=ID_VAR) %>% 
            filter(timepoint == "acute") 
    
    }
    ##------------------------------------
    # add universal id variable
    mdf_pb$sample_id <- mdf_pb[[ID_VAR]]
    
    ##------------------------------------
    # add "high" and "low" markers
    mdf_pb <- mdf_pb %>%
            mutate(group = ifelse(.data[[MARKER]] >= CUTOFF, "high","low"))
    
    return(mdf_pb)
}

paired_boxplot_publish <- function(df_melt_cf, df_melt_wb, mdf, MARKER, CUTOFF, CELLTYPE, group_palette){

    mdf_cf <- get_plot_df(mdf, df_melt_cf, MARKER, CUTOFF, "cfrna_sample_id") %>% 
        select(!cfrna_sample_id) %>%
        mutate(analyte = "cfrna")
    
    mdf_wb <- get_plot_df(mdf, df_melt_wb, MARKER, CUTOFF, "wbrna_sample_id") %>% 
        select(!wbrna_sample_id) %>%
        mutate(analyte = "wbrna")
    
    cat("\n---------------\n")
    cat(MARKER)
    cat("\ncfrna:\n")
    mdf_cf %>% filter(celltype == CELLTYPE) %>% pull(group) %>% table() %>% print()
    cat("\nwbrna:\n")
    mdf_wb %>% filter(celltype == CELLTYPE) %>% pull(group) %>% table() %>% print()
    
    mdf <- rbind(mdf_cf, mdf_wb) %>% filter(celltype == CELLTYPE) %>% unique()
    
    stat.test <- mdf %>%
    group_by(analyte) %>%
    wilcox_test(fraction ~ group, paired = FALSE) %>% 
    add_significance("p") %>%
    add_xy_position(x="analyte", dodge = 0.8)

    boxplot <- mdf %>% 
            mutate(group = factor(group, levels = c("low","high"))) %>%
            mutate(analyte = factor(analyte, levels = c("cfrna","wbrna"))) %>%
            ggplot()  + 
            geom_boxplot(aes(x=analyte,y=fraction,fill=group),outlier.shape = NA,size = 0.2, color = "black")+ #, color = "black"
            geom_point(aes(x = analyte, y= fraction, fill=group),
                       position = position_jitterdodge(jitter.width=0.2,seed=100,jitter.height=0),
                       size = 0.5)+       # ,pch=21 ,color = "black"
            theme_prevail()+
            scale_fill_manual(values = group_palette) + 
            theme(panel.grid.minor = element_blank(),
                 text = element_text(size = 18),
                 plot.margin=grid::unit(c(1,1,1,1), "mm"),
                axis.title.y = element_blank(),
                 axis.title.x = element_blank(),
                 title = element_blank())

    boxplot <- boxplot + stat_pvalue_manual(stat.test, label = "p", tip.length = 0, label.size = 2)

    return(boxplot)
}


---
# publish figures

In [11]:
MARKER = "crp"
CUTOFF = 18
CELLTYPE = "Endothelial Cell"
mdf = protein_rds[[MARKER]]
crp_plt <- paired_boxplot_publish(df_melt_cf, df_melt_wb, mdf, MARKER, CUTOFF, CELLTYPE, group_palette)
    
MARKER = "wbc"
CUTOFF = 11
CELLTYPE = 'Platelet'
mdf = protein_rds[[MARKER]]
wbc_plt <- paired_boxplot_publish(df_melt_cf, df_melt_wb, mdf, MARKER, CUTOFF, CELLTYPE, group_palette)


MARKER = "alt"
CUTOFF = 36
CELLTYPE = 'Intrahepatic cholangiocyte'
mdf = protein_rds[[MARKER]]
alt_plt <- paired_boxplot_publish(df_melt_cf, df_melt_wb, mdf, MARKER, CUTOFF, CELLTYPE, group_palette)


MARKER = "alb"
CUTOFF = 3.4
CELLTYPE = 'Kidney Epithelial Cell'
mdf = protein_rds[[MARKER]]
alb_plt <- paired_boxplot_publish(df_melt_cf, df_melt_wb, mdf, MARKER, CUTOFF, CELLTYPE, group_palette)



---------------
crp
cfrna:
.
high  low 
  18   33 

wbrna:
.
high  low 
  29   52 

---------------
wbc
cfrna:
.
high  low 
  19   29 

wbrna:
.
high  low 
  35   50 

---------------
alt
cfrna:
.
high  low 
  38   30 

wbrna:
.
high  low 
  51   60 

---------------
alb
cfrna:
.
high  low 
  12   37 

wbrna:
.
high  low 
  23   62 


In [12]:
options(repr.plot.width = 8, repr.plot.height = 6)

WIDTH = 1.55
HEIGHT = 1.28


##--------------------------
pdf(file=paste0("plots/panelF_wbc-platelet.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

YLIM = 1.2
wbc_plt + scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM)) + coord_cartesian(ylim = c(0,YLIM*1.05))

dev.off()
##--------------------------
pdf(file=paste0("plots/panelF_crp-endothelial.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

YLIM = 0.6
crp_plt + scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM)) + coord_cartesian(ylim = c(0,YLIM*1.05))

dev.off()

WIDTH = 1.64
##--------------------------
pdf(file=paste0("plots/panelF_alb-kidney.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

YLIM = 0.021
alb_plt + scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM)) + coord_cartesian(ylim = c(0,YLIM*1.05))

dev.off()
##--------------------------
pdf(file=paste0("plots/panelF_alt-liver.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

YLIM = 0.042
alt_plt + scale_y_continuous(breaks = c(0, YLIM/3, (YLIM/3)*2 , YLIM)) + coord_cartesian(ylim = c(0,YLIM*1.05))

dev.off()

png 
  2

png 
  2

png 
  2

png 
  2